In [1]:
from utils import load_data, check_gpu, check_dir
from clean_session import preprocess_sessions
import pandas as pd
import numpy as np
from tqdm import tqdm
import gc
from functools import partial
import matplotlib.pyplot as plt
from clean_session import preprocess_sessions
from create_model_inputs import prepare_data, compute_session_fts, create_model_inputs
import datetime
%matplotlib inline
%load_ext autoreload
%autoreload 2
def fprint(df, name):
    print(f'{name} shape: ({df.shape[0]:,}, {df.shape[1]})')
import pprint
pp = pprint.PrettyPrinter(indent=1)

In [ ]:
nrows = 1000000
train = prepare_data('train', convert_action_type=False, nrows=nrows, recompute=True)
df = compute_session_fts(train, 'train')

[05-17 14:35:34 - utils-106 - load_data - INFO] Loading train using 1,000,000 rows (999,995 trimmed) which is 6.28% out of total train data
[05-17 14:35:34 - create_model_inputs-17 - flogger - INFO] raw train shape: (999,995, 12)
[05-17 14:35:34 - clean_session-60 - preprocess_sessions - INFO] Dropping duplicates
[05-17 14:35:34 - clean_session-18 - remove_duplicates - INFO] Before dropping duplicates df shape: (999,995, 12)
[05-17 14:35:35 - clean_session-21 - remove_duplicates - INFO] After dropping duplicates df shape: (421,117, 12)
[05-17 14:35:35 - clean_session-62 - preprocess_sessions - INFO] Cliping session dataframe up to last click out (if there is clickout)
[05-17 14:36:01 - clean_session-65 - preprocess_sessions - INFO] Filtering out sessions without clickouts, reference, or clickout is nan
[05-17 14:36:01 - clean_session-66 - preprocess_sessions - INFO] train length before filtering no click-out, or nan reference and click-out sessions: 362,672


In [ ]:
df = df.groupby('session_id').last().reset_index()
# df['session_id_size'] = np.log(df['session_id_size'])
# df['cfs'] = df['current_filters_last_filters'].str.lower().str.split('|')

# df['prices'] = df['prices'].str.split('|')
# df['prices'] = df['prices'].apply(lambda x: [int(p) for p in x])

# df['time_steps'] = df['prices'].str.len()
# padding_mask = df['time_steps'] < 25
# df.loc[padding_mask, 'prices'] = df.loc[padding_mask, 'prices'].apply(lambda x: np.pad(x, (0, 25-len(x)),
#                                                                                        mode='constant'))
df['imps'] = df['impressions'].str.split('|')
df['imps'] = df['imps'].apply(lambda x: [int(i) for i in x])
# df.loc[padding_mask, 'impressions'] = (df.loc[padding_mask, 'impressions']
#                                          .apply(lambda x: np.pad(x, (0, 25-len(x)),mode='constant')))
df['reference'] = df['reference'].astype(int)
# filter out nan rows with reference_id not in impressions list, since if the true target in test
# is not in the impression list then it would not get evaluated
def assign_target(row):
    ref = row['reference']
    imp = list(row['imps'])
    if ref in imp:
        return imp.index(ref)
    else:
        return np.nan

df['target'] = df.apply(assign_target, axis=1)
# drop the ones whose reference is not in the impression list
df = df[df['target'].notna()].reset_index(drop=True)
df['target'] = df['target'].astype(int)
def assign_last_ref_id(row, divide=True):
    ref = row['reference_last_reference_id']
    # although reference_id got converted to int, but the reference_last_reference_id was calculated
    # when it was still str value, so here we look up the index in str of impressions
    imp = [str(i) for i in row['imps']]

    if pd.isna(ref):
        return np.nan
    else:
        if ref in imp:
            if divide:
                return (imp.index(ref) + 1) / len(imp)
            else:
                return imp.index(ref) + 1
        else:
            return np.nan
assign_last_ref_id_func = partial(assign_last_ref_id, divide=False)
df['last_ref_ind'] = df.apply(assign_last_ref_id_func, axis=1)

### comapre session with size 1 and the rest

In [ ]:
df.head(2)

In [ ]:
ones = df[df['session_id_size']==1].reset_index(drop=True)
multiples = df[df['session_id_size']>1].reset_index(drop=True)

In [ ]:
oids = ones['session_id'].unique()
mids = multiples['session_id'].unique()

---
### ones

In [ ]:
display_cols = ['session_id', 'step', 'action_type', 'current_filters', 'reference', 'timestamp',
                'impressions', 'prices', 'target', 'session_id_size']
raw_display_cols = [c for c in train.columns if c in display_cols]

In [ ]:
# pd.options.display.max_colwidth = 100

In [ ]:
one_rid = np.random.choice(oids, 1)[0]
print(one_rid)
df[df.session_id==one_rid][display_cols].reset_index(drop=True)

In [ ]:
pp.pprint(df[df.session_id==one_rid][display_cols].reset_index(drop=True).to_dict(orient='list'))

In [ ]:
# and the raw
train[train.session_id==one_rid][raw_display_cols].reset_index(drop=True)

In [ ]:
pp.pprint(train[train.session_id==one_rid][raw_display_cols].reset_index(drop=True).to_dict(orient='list'))

---
### Longer

In [ ]:
mid_rid = np.random.choice(mids, 1)[0]
# df[df.session_id==mid_rid][display_cols].reset_index(drop=True)

In [ ]:
np.sort([int(i) for i in df[df.session_id==mid_rid]['prices'].str.split('|').values[0]])

In [ ]:
# pp.pprint(df[df.session_id==mid_rid][display_cols].reset_index(drop=True).to_dict(orient='list'))

In [ ]:
# the raw
train[train.session_id==mid_rid][raw_display_cols].reset_index(drop=True)

In [ ]:
pp.pprint(train[train.session_id==mid_rid][raw_display_cols].reset_index(drop=True).to_dict(orient='list'))

In [ ]:
# dup_mask = train[[c for c in train.columns if c not in ['timestamp', 'step']]].duplicated(keep=False)
# dup_mask.sum()

In [ ]:
# dup_cols = [c for c in train.columns if c not in ['timestamp', 'step']]
# # train[dup_cols].loc[train[dup_cols].shift(-1) != train[dup_cols]]
# train.groupby(dup_cols).apply(lambda x: )

In [ ]:
train.shape

In [ ]:
99508/train.shape[0]

### Click-encodings

In [ ]:
from create_model_inputs import click_view_encoding
_ = click_view_encoding(sids=None, fold='testing', m=5)

In [ ]:
cv_enc = pd.read_csv('./cache/foldtesting_clickview_encodings.csv')

In [ ]:
cv_enc.shape

In [ ]:
cv_enc['clicked'].hist(bins=100)